In [15]:
import requests
import pandas as pd
import xml.etree.ElementTree as et


def get_request():
    url="https://api.n11.com/ws/ProductService.wsdl"
    #headers = {'content-type': 'application/soap+xml'}
    headers = {'content-type': 'text/xml'}
    body = """<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:sch="http://www.n11.com/ws/schemas">
       <soapenv:Header/>
       <soapenv:Body>
          <sch:GetProductListRequest>
             <auth>
                <appKey>api-key-field</appKey>
                <appSecret>api-secret-key-field</appSecret>
             </auth>
             <pagingData>
                <currentPage>0</currentPage>
                <pageSize>100</pageSize>
             </pagingData>
          </sch:GetProductListRequest>
       </soapenv:Body>
    </soapenv:Envelope>"""
    response = requests.post(url,data=body,headers=headers)
    print(response.content)
    
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

df_cols =['products','product','title','subTitle','currencyAmount','currencyType','displayPrice','isDomestic','id','price','productSellerCode','approvalStatus','saleStatus','stockItems','bundle','currencyAmount','displayPrice','optionPrice','attributes','id','quantity','version'] 
df=parse_XML('xml_file.xml',df_cols=df_cols)

In [16]:
get_request()

b'<env:Envelope xmlns:env="http://schemas.xmlsoap.org/soap/envelope/"><env:Header/><env:Body><ns3:GetProductListResponse xmlns="" xmlns:ns3="http://www.n11.com/ws/schemas"><result><status>success</status></result><products><product><currencyAmount>250.00</currencyAmount><currencyType>1</currencyType><displayPrice>192.50</displayPrice><isDomestic>false</isDomestic><id>392590007</id><price>250.00</price><productSellerCode>90021</productSellerCode><approvalStatus>1</approvalStatus><saleStatus>2</saleStatus><stockItems><stockItem><bundle>false</bundle><currencyAmount>250.00</currencyAmount><displayPrice>192.50</displayPrice><optionPrice>250.00</optionPrice><attributes/><id>126757920343</id><quantity>5</quantity><version>1</version></stockItem></stockItems><subtitle>40 Kg Elektronik Terazi Manav Terazisi Bakkal Terazisi</subtitle><title>40 Kg Elektronik Terazi Manav Terazisi Bakkal Terazisi</title></product><product><currencyAmount>420.00</currencyAmount><currencyType>1</currencyType><displ

In [19]:
df.to_csv('product.csv')